In [22]:
import pandas as pd
import random
import json

# Generating Synthetic Dataset for Definition intent

### Manually added question formats

In [23]:
ques_formats_pre = [
    'what is ',
    'summary of ',
    'i want summary of ',
    'brief explanation about ',
    'i want to know more about ',
    'summary for the current topic ',
    'more information about ',
    'summary for ',
    'give me summary for ',
    'i want summary for ',
    'what does ',
    'tell me something about ',
    'explain ',
    'elaborate ',
    'can you tell me about ',
    'what do you know about ',
    'what can you tell me about ',
    'do you have information about ',
    'define ',
    'can you define ',
    'give me definition of ',
    # New questions
    'may i please have a definition for ',
    'what is the meaning of ',
    'please define ',
    'tell me the meaning of ',
    'define ',
    'i need to know what ',
    'tell me please what ',
    'what is the definition of ',
    'exactly what does ',
    'tell me what the word ',
    'tell me the definition of ',
    'can you give the definition of ',
    'please share the definition of ',
    '',
    'find what the word ',
    'do you know what ',
    'give me the meaning of ',
    'i would like to know what ',
    'can you tell me what ',
    'i do not undertand ',
    'i cannot understand ',
    'i am unable to understand ',
    'please explain ',
]

ques_formats_suff = [
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    ' mean',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    # New questions
    '',
    '',
    '',
    '',
    ' for me please',
    ' means',
    ' means',
    '',
    ' mean',
    ' means',
    '',
    '',
    '',
    ' means what',
    ' mean please',
    ' means',
    '',
    ' means',
    ' means',
    '',
    '',
    '',
    '',
]


if(len(ques_formats_pre) != len(ques_formats_suff)):
    print("Length of question format pre != Length of question format suff")

print(f"Number of question formats = {len(ques_formats_pre)}")

In [24]:
keywords = []
with open('../input/keywords/keywords.txt','r') as file1:
    currs = file1.readlines()
    for curr in currs:
        keywords.append(curr.strip())

print(f"Length of keywords.txt = {len(keywords)}")

In [25]:
def extractKeywords(keywords, max_n_gram, max_keywords_per_ngram, seed = 5):
    ngrams = [[] for _ in range(max_n_gram)] # Storing unigram, bigram and trigrams, etc

    # segregating unigram, bigram and trigrams 
    for keyword in keywords:
        ngram = len(keyword.split())
        if(ngram > max_n_gram or ngram < 1):
            continue
        ngrams[ngram - 1].append(keyword)

    final_keywords = []

    random.seed(seed)
    for i in range(max_n_gram):
        sample_list = random.sample(ngrams[i], max_keywords_per_ngram)
        final_keywords.extend(sample_list)

    print(f"Length of final keywords selected = {len(final_keywords)}")

    return final_keywords

In [26]:
final_keywords = extractKeywords(keywords, max_n_gram=3, max_keywords_per_ngram=150, seed=5)

In [27]:
lst1 = []
lst2 = []

for keyword in final_keywords:
    for i in range(len(ques_formats_pre)):
        pre = ques_formats_pre[i]
        suff = ques_formats_suff[i]
        lst1.append(pre+keyword+suff)
        lst2.append('definition')

In [28]:
defintion_df = pd.DataFrame()
defintion_df['Text'] = lst1
defintion_df['Intent'] = lst2

print(len(lst1))

# Extracting CLINC150 Dataset defintions and OOS

In [29]:
with open('../input/intentclftest/data_full.json') as jsondata:
    dataset = json.load(jsondata)

In [30]:
df = []
for ky in dataset.keys():
    print(ky)
    df += dataset[ky]
df = pd.DataFrame(df, columns = ['Text', 'Intent']).iloc[:]

In [31]:
print(len(df))

In [32]:
# df['Intent'].unique()

In [33]:
check = 'definition'
df.loc[df['Intent'] == check].head()

In [34]:
print(len(df.loc[df['Intent'] == check]))

In [35]:
filtered_definitons_clinc150 = df.loc[df['Intent'].isin(['definition'])]

In [36]:
len(filtered_definitons_clinc150)

In [37]:
filtered_oos_clinc150 = df.loc[df['Intent'].isin(['definition']) ==  False]

In [38]:
len(filtered_oos_clinc150)

In [39]:
# Converting all other intents to 'oos'
filtered_oos_clinc150 = filtered_oos_clinc150.drop(['Intent'], axis = 1)
filtered_oos_clinc150.insert(len(filtered_oos_clinc150.columns), 'Intent', 'oos')

In [40]:
final_df = pd.concat([defintion_df, filtered_definitons_clinc150, filtered_oos_clinc150])

In [41]:
len(final_df)

In [42]:
final_df.to_csv('./IntentDataset.csv')